# Set Up

In [ ]:
import tensorflow as tf
import random
import numpy as np
import os
from google.colab import drive

In [ ]:
print(tf.__version__)

2.12.0


In [ ]:
drive.mount('/content/gdrive/', force_remount=True)
%cd '/content/gdrive/MyDrive/University/U of T/4th Year/ECE496 Team/Hardware/_WORKING_VERSION_'
!ls

Mounted at /content/gdrive/
/content/gdrive/MyDrive/University/U of T/4th Year/ECE496 Team/Hardware/_WORKING_VERSION_
best_128CNNsBiLSTMmodel		    mcu_data		 mcu_eeg_data1_1.txt
best_doubleCNNmodel		    mcu_data_batch	 mcu_eeg_data1_2.txt
best_doubleCNNmodel.cc		    mcu_eeg_data0_0.txt  mcu_eeg_data1_3.txt
best_doubleCNNmodel.tflite	    mcu_eeg_data0_1.txt  mcu_eeg_data1_4.txt
Convert_New_Model.ipynb		    mcu_eeg_data0_2.txt  testdata
lfinalbest_doubleCNNmodel_3	    mcu_eeg_data0_3.txt  testdata20230405
lfinalbest_doubleCNNmodel_3.cc	    mcu_eeg_data0_4.txt  Videos
lfinalbest_doubleCNNmodel_3.tflite  mcu_eeg_data1_0.txt


# Data Processing

In [ ]:
# Label values
W = 0
N1 = 1
N2 = 2
N3 = 3
REM = 4
UNKNOWN = 5

NUM_CLASSES = 6  # exclude UNKNOWN

class_dict = {
    0: "W",
    1: "N1",
    2: "N2",
    3: "N3",
    4: "REM",
    5: "?"
}

EPOCH_SEC_LEN = 30  # seconds
SAMPLING_RATE = 256

def print_n_samples_each_class(labels):
    
    unique_labels = np.unique(labels)
    for c in unique_labels:
        n_samples = len(np.where(labels == c)[0])
        # print("{}: {}".format(class_dict[c], n_samples))

In [ ]:
# import tensorflow as tf
# from tensorflow import keras

def load_npz_file(npz_file):
    """Load data and labels from a npz file."""
    with np.load(npz_file) as f:
        data = f["x"]
        labels = f["y"]
        sampling_rate = f["fs"]
    return data, labels, sampling_rate
    
def load_npz_list_files(data_dir, npz_files):
    """Load data and labels from list of npz files."""
    data = []
    labels = []
    fs = None
    for npz_f in npz_files:
        # print("Loading {} ...".format(npz_f))
        # print("npz_f.....",npz_f)
        npz_file = data_dir + '/' + npz_f
        tmp_data, tmp_labels, sampling_rate = load_npz_file(npz_file)
        
        if fs is None:
            fs = sampling_rate
        elif fs != sampling_rate:
            raise Exception("Found mismatch in sampling rate.")
        data.append(tmp_data)
        labels.append(tmp_labels)

    data = np.vstack(data)
    labels = np.hstack(labels)
    return data, labels


def get_balance_class_oversample(x, y):
    """
    Balance the number of samples of all classes by (oversampling):
        1. Find the class that has the largest number of samples
        2. Randomly select samples in each class equal to that largest number
    """
    class_labels = np.unique(y)
    n_max_classes = -1
    for c in class_labels:
        n_samples = len(np.where(y == c)[0])
        if n_max_classes < n_samples:
            n_max_classes = n_samples

    balance_x = []
    balance_y = []
    for c in class_labels:

        idx = np.where(y == c)[0]
        # print('c',c)
        # print('idx',idx)
        n_samples = len(idx)
        n_repeats = int(n_max_classes / n_samples)
        # print('n_repeats',n_repeats)
        # print('x',x)
        # print('len(x)',len(x))
        # print('x[idx]',x[idx])
        tmp_x = np.repeat(x[idx], n_repeats, axis=0)
        tmp_y = np.repeat(y[idx], n_repeats, axis=0)
        n_remains = n_max_classes - len(tmp_x)
        if n_remains > 0:
            sub_idx = np.random.permutation(idx)[:n_remains]
            tmp_x = np.vstack([tmp_x, x[sub_idx]])
            tmp_y = np.hstack([tmp_y, y[sub_idx]])
        balance_x.append(tmp_x)
        balance_y.append(tmp_y)
    balance_x = np.vstack(balance_x)
    balance_y = np.hstack(balance_y)

    return balance_x, balance_y

In [ ]:
def preprocess_data(data_dir, trainfile_idx, testfile_idx):
    # Load all files
    print('data_dir:',data_dir)
    allfiles = os.listdir(data_dir)
    print("allfiles",allfiles)
    train_files = []
    test_files = []
    
    for idx in trainfile_idx:
       
        train_files.append(allfiles[idx])
        
    for idx in testfile_idx:
        test_files.append(allfiles[idx])
    # # Ns: the number of subjects in the dataset
    # Ns = int(len(allfiles) / k_folds)
    # print('Ns:',Ns)
    
    # Split train and test files
    test_files = sorted(test_files)
    train_files = sorted(train_files)
    
    # print("test_files:",test_files)
    
    # Load data in npz files
    # data_train: (2796, 1, 7680)
    # label_train: (2796,)
    data_train, label_train = load_npz_list_files(data_dir,train_files)
    # x_val: (2884, 1, 7680)
    # y_val: (2884,)
    x_val, y_val = load_npz_list_files(data_dir, test_files)
    # print('data_train',data_train.shape) # 
    # print('x_val',x_val.shape) # (2884, 1, 7680)
    # print('label_train',label_train.shape) # (2796,)
    # print('y_val',y_val.shape) # (2884,)
    
    # Reshape the data to match the input of the model - conv2d
    data_train = np.squeeze(data_train) # (3868, 7680)
    x_val = np.squeeze(x_val) # (1812, 7680)
    data_train = data_train[:, :, np.newaxis, np.newaxis] # (3868, 7680, 1, 1)
    x_val = x_val[:, :, np.newaxis, np.newaxis] # (1812, 7680, 1, 1)
    
    # Casting
    data_train = data_train.astype(np.float32)
    label_train = label_train.astype(np.int32)
    x_val = x_val.astype(np.float32)
    y_val = y_val.astype(np.int32)
        
    # print('reshaped data_train',data_train.shape)
    # print('reshaped x_val',x_val.shape)
    
    # print("Training set: {}, {}".format(data_train.shape, label_train.shape))
    # print_n_samples_each_class(label_train)
    # print(" ")
    # print("Validation set: {}, {}".format(x_val.shape, y_val.shape))
    # print_n_samples_each_class(y_val)
    # print(" ")
    
    # Use balanced-class, oversample training set
    x_train, y_train = get_balance_class_oversample(
            x=data_train, y=label_train
    )
    # print("Oversampled training set: {}, {}".format(
    #     x_train.shape, y_train.shape
    # ))
    print_n_samples_each_class(y_train)
    print(" ")
    return x_train, y_train, x_val, y_val

#Convert

In [ ]:
def representative_data_gen():

  #trainfile_idx = [0,3,4,5]
  #testfile_idx = [1,2]
  trainfile_idx = [0,1,2,3]
  testfile_idx = [4,5]
  data_dir = "/content/gdrive/MyDrive/University/U of T/4th Year/ECE496 Team/Software/ECE496Software/data/eeg_fz_ler"

  x_train, y_train, x_val, y_val = preprocess_data(data_dir, trainfile_idx, testfile_idx)
  test_batches_ori = (x_val, y_val)
  # input_value = x_val[:1, :]
  # print(input_value.shape)
  # for i in range(500):
  #   print(x_val[i-1:i+1, :].shape)
  #   yield([x_val[i-1:i+1, :]])
  # for input_value, _ in test_batches.take(100):
  #   yield [input_value]
  #yield input_value
  x_val = x_val.swapaxes(1, 2)
  for i in range(101):
  #   yield [input_value]
    #print(x_val[i:i+1, :].shape)
    yield [x_val[i:i+1, :]]


In [ ]:
export_dir = 'best_128CNNsBiLSTMmodel'
tflite_model_file = 'best_128CNNsBiLSTMmodel.tflite'

converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Enforce integer only quantization
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
#converter.inference_input_type = tf.int8
#converter.inference_output_type = tf.int8

# For BiLSTM
converter._experimental_default_to_single_batch_in_tensor_list_ops = True
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
#converter._experimental_lower_tensor_list_ops = False

# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

with open(tflite_model_file, "wb") as f:
  f.write(tflite_model)

data_dir: /content/gdrive/MyDrive/University/U of T/4th Year/ECE496 Team/Software/ECE496Software/data/eeg_fz_ler
allfiles ['01-03-0001.npz', '01-03-0002.npz', '01-03-0003.npz', '01-03-0004.npz', '01-03-0012.npz', '01-03-0008.npz']
 


In [ ]:
!xxd -i 'best_128CNNsBiLSTMmodel.tflite' > 'best_128CNNsBiLSTMmodel.cc'

# TFLITE Analyzer

In [ ]:
tf.lite.experimental.Analyzer.analyze(model_path='best_128CNNsBiLSTMmodel.tflite')

=== best_128CNNsBiLSTMmodel.tflite ===

Your TFLite model has '9' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the QUANTIZE op takes
tensor #0 as input and produces tensor #70 as output.

Subgraph#0 main(T#0) -> [T#243]
  Op#0 QUANTIZE(T#0) -> [T#70]
  Op#1 EXPAND_DIMS(T#70, T#3[-3]) -> [T#71]
  Op#2 RESHAPE(T#71, T#4[-1, 1, 7680, 1]) -> [T#72]
  Op#3 CONV_2D(T#72, T#69, T#67[0, 0, 0, 0, 0, ...]) -> [T#73]
  Op#4 SHAPE(T#71) -> [T#74]
  Op#5 STRIDED_SLICE(T#74, T#5[0], T#6[-3], T#7[1]) -> [T#75]
  Op#6 CONCATENATION(T#75, T#8[1, 1280, 64]) -> [T#76]
  Op#7 RESHAPE(T#73, T#76) -> [T#77]
  Op#8 SQUEEZE(T#77) -> [T#78]
  Op#9 MUL(T#78, T#66) -> [T#79]
  Op#10 ADD(T#79, T#65) -> [T#80]
  Op#11 MAX_POOL_2D(T#80) -> [T#81]
  Op#12 EXPAND_DIMS(T#81, T#3[-3]) -> [T#82]
  Op#13 RESHAPE(T#82, T#9[-1, 1, 160, 64]) -> [T#83]
  Op#14 CONV_2D(T#83, T#64, T#58[0, 0, 0, 0, 0, ...]) -> [T#84]
  Op#15 SHAPE(T#82) -> [T#85]
  Op#16 STRIDED_

In [ ]:
tf.lite.experimental.Analyzer.analyze(model_path='ltrybest_doubleCNNmodel.tflite')

=== ltrybest_doubleCNNmodel.tflite ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the QUANTIZE op takes
tensor #0 as input and produces tensor #59 as output.

Subgraph#0 main(T#0) -> [T#175]
  Op#0 QUANTIZE(T#0) -> [T#59]
  Op#1 EXPAND_DIMS(T#59, T#1[-3]) -> [T#60]
  Op#2 RESHAPE(T#60, T#2[-1, 1, 7680, 1]) -> [T#61]
  Op#3 CONV_2D(T#61, T#58, T#56[0, 0, 0, 0, 0, ...]) -> [T#62]
  Op#4 SHAPE(T#60) -> [T#63]
  Op#5 STRIDED_SLICE(T#63, T#3[0], T#4[-3], T#5[1]) -> [T#64]
  Op#6 CONCATENATION(T#64, T#6[1, 2560, 8]) -> [T#65]
  Op#7 RESHAPE(T#62, T#65) -> [T#66]
  Op#8 SQUEEZE(T#66) -> [T#67]
  Op#9 MUL(T#67, T#55) -> [T#68]
  Op#10 ADD(T#68, T#54) -> [T#69]
  Op#11 MAX_POOL_2D(T#69) -> [T#70]
  Op#12 EXPAND_DIMS(T#70, T#1[-3]) -> [T#71]
  Op#13 RESHAPE(T#71, T#7[-1, 1, 640, 8]) -> [T#72]
  Op#14 CONV_2D(T#72, T#53, T#47[0, 0, 0, 0, 0, ...]) -> [T#73]
  Op#15 SHAPE(T#71) -> [T#74]
  Op#16 STRIDED_SL

In [ ]:
tf.lite.experimental.Analyzer.analyze(model_path='lfinalbest_doubleCNNmodel_3.tflite')

=== lfinalbest_doubleCNNmodel_3.tflite ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the QUANTIZE op takes
tensor #0 as input and produces tensor #59 as output.

Subgraph#0 main(T#0) -> [T#175]
  Op#0 QUANTIZE(T#0) -> [T#59]
  Op#1 EXPAND_DIMS(T#59, T#1[-3]) -> [T#60]
  Op#2 RESHAPE(T#60, T#2[-1, 1, 7680, 1]) -> [T#61]
  Op#3 CONV_2D(T#61, T#58, T#56[0, 0, 0, 0, 0, ...]) -> [T#62]
  Op#4 SHAPE(T#60) -> [T#63]
  Op#5 STRIDED_SLICE(T#63, T#3[0], T#4[-3], T#5[1]) -> [T#64]
  Op#6 CONCATENATION(T#64, T#6[1, 2560, 8]) -> [T#65]
  Op#7 RESHAPE(T#62, T#65) -> [T#66]
  Op#8 SQUEEZE(T#66) -> [T#67]
  Op#9 MUL(T#67, T#55) -> [T#68]
  Op#10 ADD(T#68, T#54) -> [T#69]
  Op#11 MAX_POOL_2D(T#69) -> [T#70]
  Op#12 EXPAND_DIMS(T#70, T#1[-3]) -> [T#71]
  Op#13 RESHAPE(T#71, T#7[-1, 1, 640, 8]) -> [T#72]
  Op#14 CONV_2D(T#72, T#53, T#47[0, 0, 0, 0, 0, ...]) -> [T#73]
  Op#15 SHAPE(T#71) -> [T#74]
  Op#16 STRIDE

Works but has error message while predicting

In [ ]:
tf.lite.experimental.Analyzer.analyze(model_path='ltestbest_2CNNsBiLSTMmodel_2.tflite')

=== ltestbest_2CNNsBiLSTMmodel_2.tflite ===

Your TFLite model has '5' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the QUANTIZE op takes
tensor #0 as input and produces tensor #71 as output.

Subgraph#0 main(T#0) -> [T#219]
  Op#0 QUANTIZE(T#0) -> [T#71]
  Op#1 EXPAND_DIMS(T#71, T#3[-3]) -> [T#72]
  Op#2 RESHAPE(T#72, T#4[-1, 1, 7680, 1]) -> [T#73]
  Op#3 CONV_2D(T#73, T#70, T#68[0, 0, 0, 0, 0, ...]) -> [T#74]
  Op#4 SHAPE(T#72) -> [T#75]
  Op#5 STRIDED_SLICE(T#75, T#5[0], T#6[-3], T#7[1]) -> [T#76]
  Op#6 CONCATENATION(T#76, T#8[1, 2560, 8]) -> [T#77]
  Op#7 RESHAPE(T#74, T#77) -> [T#78]
  Op#8 SQUEEZE(T#78) -> [T#79]
  Op#9 MUL(T#79, T#67) -> [T#80]
  Op#10 ADD(T#80, T#66) -> [T#81]
  Op#11 MAX_POOL_2D(T#81) -> [T#82]
  Op#12 EXPAND_DIMS(T#82, T#3[-3]) -> [T#83]
  Op#13 RESHAPE(T#83, T#9[-1, 1, 640, 8]) -> [T#84]
  Op#14 CONV_2D(T#84, T#65, T#59[0, 0, 0, 0, 0, ...]) -> [T#85]
  Op#15 SHAPE(T#83) -> [T#86]
  Op#16 STRID

# Data Generation

## Test

In [ ]:
def test_preprocess_data(data_dir):
    # Load all files
    # print('data_dir:',data_dir)
    test_files = os.listdir(data_dir)
    
    test_files = sorted(test_files)
    
    x_val, y_val = load_npz_list_files(data_dir, test_files)
    # print('data_train',data_train.shape) # 
    # print('x_val',x_val.shape) # (2884, 1, 7680)
    # print('label_train',label_train.shape) # (2796,)
    # print('y_val',y_val.shape) # (2884,)
    
    # Reshape the data to match the input of the model - conv2d
    x_val = np.squeeze(x_val) # (1812, 7680)
    x_val = x_val[:, :, np.newaxis, np.newaxis] # (1812, 7680, 1, 1)
    
    # Casting
    x_val = x_val.astype(np.float32)
    y_val = y_val.astype(np.int32)
        
    print(" ")
    return x_val, y_val

In [ ]:
x_test, y_test = test_preprocess_data("testdata20230405")
x_test = x_test.swapaxes(1, 2)

idx = np.random.permutation(len(x_test))
x_test,y_test = x_test[idx], y_test[idx]

for i in range(len(y_test)):
  print("[",i,"]=",y_test[i],end="  ")

 
[ 0 ]= 4  [ 1 ]= 2  [ 2 ]= 2  [ 3 ]= 0  [ 4 ]= 1  [ 5 ]= 2  [ 6 ]= 0  [ 7 ]= 1  [ 8 ]= 4  [ 9 ]= 2  [ 10 ]= 4  [ 11 ]= 2  [ 12 ]= 2  [ 13 ]= 2  [ 14 ]= 2  [ 15 ]= 1  [ 16 ]= 2  [ 17 ]= 2  [ 18 ]= 2  [ 19 ]= 2  [ 20 ]= 2  [ 21 ]= 2  [ 22 ]= 1  [ 23 ]= 2  [ 24 ]= 3  [ 25 ]= 2  [ 26 ]= 3  [ 27 ]= 3  [ 28 ]= 2  [ 29 ]= 2  [ 30 ]= 2  [ 31 ]= 3  [ 32 ]= 3  [ 33 ]= 0  [ 34 ]= 2  [ 35 ]= 2  [ 36 ]= 2  [ 37 ]= 4  [ 38 ]= 2  [ 39 ]= 2  [ 40 ]= 2  [ 41 ]= 4  [ 42 ]= 2  [ 43 ]= 0  [ 44 ]= 4  [ 45 ]= 2  [ 46 ]= 2  [ 47 ]= 1  [ 48 ]= 2  [ 49 ]= 2  [ 50 ]= 1  [ 51 ]= 2  [ 52 ]= 2  [ 53 ]= 0  [ 54 ]= 2  [ 55 ]= 2  [ 56 ]= 0  [ 57 ]= 1  [ 58 ]= 2  [ 59 ]= 1  [ 60 ]= 4  [ 61 ]= 2  [ 62 ]= 4  [ 63 ]= 4  [ 64 ]= 1  [ 65 ]= 0  [ 66 ]= 0  [ 67 ]= 2  [ 68 ]= 2  [ 69 ]= 0  [ 70 ]= 0  [ 71 ]= 4  [ 72 ]= 4  [ 73 ]= 2  [ 74 ]= 4  [ 75 ]= 2  [ 76 ]= 1  [ 77 ]= 2  [ 78 ]= 3  [ 79 ]= 2  [ 80 ]= 2  [ 81 ]= 2  [ 82 ]= 2  [ 83 ]= 2  [ 84 ]= 4  [ 85 ]= 1  [ 86 ]= 1  [ 87 ]= 2  [ 88 ]= 2  [ 89 ]= 3  [ 90 ]= 2  [ 91 ]=

## Save to File

In [ ]:
import sys

In [ ]:
# save directly to file instead
def data_gen_mcu():
    sample_data = "mcu_eeg_data1_4.txt"
    original_stdout = sys.stdout

    with open(sample_data, "w") as f:
      sys.stdout = f
      print(y_test[1576])
      want_data = x_test[1576:1576+1, :]
      for j in range(7680):
          print('{:.15f}'.format(want_data[0][0][j][0]))

    sys.stdout = original_stdout

In [ ]:
data_gen_mcu()

In [ ]:
# save directly to file instead
import sys

def data_gen():
  sample_data = "mcu_data_0.txt"
  original_stdout = sys.stdout

  with open(sample_data, "w") as f:
    sys.stdout = f

    """Label
    print("float sleep_label[2] = {", end="")
    for i in range(2):
      print(y[i], end="")
      if i != 1:
        print(",", end="")
    print("};")"""

    #Data
    print("float sleep_array[7680] = {", end="")

    #for i in range(2):
      #print("{", end="")
    want_data = x_val[0:1, :]
    for j in range(7680):
      if j % 10 == 0:
        print()
      if j == 7679:
        print(want_data[0][0][j][0], end="")
      else:
        print(want_data[0][0][j][0], end=",")
    #print("}", end="")
      #if i != 1:
        #print(",", end="")

    print("};")

  sys.stdout = original_stdout

In [ ]:
data_gen()

## Train & Val

In [ ]:
#trainfile_idx = [0,3,4,5]
#testfile_idx = [1,2]
trainfile_idx = [0,1,2,3]
testfile_idx = [4,5]

data_dir = "/content/gdrive/MyDrive/University/U of T/4th Year/ECE496 Team/Software/ECE496Software/data/eeg_fz_ler"

x_train, y_train, x_val, y_val = preprocess_data(data_dir, trainfile_idx, testfile_idx)
test_batches_ori = (x_val, y_val)
x_train = x_train.swapaxes(1, 2)
x_val = x_val.swapaxes(1, 2)

data_dir: /content/gdrive/MyDrive/University/U of T/4th Year/ECE496 Team/Software/ECE496Software/data/eeg_fz_ler
allfiles ['01-03-0001.npz', '01-03-0002.npz', '01-03-0003.npz', '01-03-0004.npz', '01-03-0012.npz', '01-03-0008.npz']
 


In [ ]:
# NEED TO SCRAMBLE THE DATA
# https://stackoverflow.com/questions/43229034/randomly-shuffle-data-and-labels-from-different-files-in-the-same-order

idx = np.random.permutation(len(x_train))
x_train,y_train = x_train[idx], y_train[idx]

for label in y_train:
  print(label,end=" ")

1 1 0 0 3 2 4 3 4 1 2 0 0 2 1 3 0 2 0 0 3 4 0 2 0 4 4 4 0 2 1 0 4 1 1 2 4 1 2 0 1 3 1 3 4 4 0 2 2 3 4 2 1 1 2 2 2 0 1 2 0 2 2 1 2 0 1 2 2 3 1 3 2 1 4 0 3 3 3 3 4 1 0 1 0 2 1 4 0 0 1 1 1 4 4 4 1 4 3 4 0 4 3 4 3 0 4 4 3 2 4 1 4 1 0 1 4 0 1 1 0 0 3 3 1 2 3 2 0 2 0 2 1 1 2 4 2 4 2 2 4 4 2 0 0 1 1 2 4 2 2 0 1 1 0 2 0 0 1 1 0 0 4 3 1 2 2 4 1 4 4 4 0 2 2 4 4 4 2 2 0 4 0 4 3 0 0 4 1 4 1 0 2 3 4 3 0 0 3 2 0 4 3 2 4 1 0 3 2 0 3 0 2 0 3 4 4 0 0 2 4 0 4 3 3 1 3 1 1 0 3 3 3 3 2 2 1 4 0 3 0 0 1 4 1 0 3 4 4 4 2 0 3 2 2 1 3 2 0 3 2 4 1 3 0 1 4 3 3 4 4 4 1 4 1 2 4 1 3 2 4 4 0 4 1 3 3 4 0 1 2 3 3 4 1 3 2 3 3 4 2 0 3 0 3 0 1 0 4 3 3 0 1 2 3 4 4 2 1 0 1 4 3 1 0 4 4 3 2 1 1 1 3 3 3 0 4 3 2 4 2 2 4 0 0 3 0 4 3 0 1 4 1 1 4 3 4 0 2 0 3 0 0 2 0 4 3 3 2 4 1 3 3 2 0 1 1 4 2 4 3 3 3 4 0 0 0 0 1 0 3 2 0 2 0 2 1 0 3 1 0 3 4 4 0 1 4 3 4 2 1 4 2 3 0 3 4 2 2 0 2 0 2 1 4 0 1 3 4 3 4 2 3 4 1 1 4 0 1 0 3 4 4 4 4 0 3 2 0 0 0 4 2 0 2 0 0 4 0 4 3 0 3 1 1 2 4 4 0 0 1 3 3 4 3 3 1 4 2 0 4 1 0 0 4 3 2 4 0 0 4 4 0 4 3 3 4 2 4 0 

In [ ]:
# NEED TO SCRAMBLE THE DATA
# https://stackoverflow.com/questions/43229034/randomly-shuffle-data-and-labels-from-different-files-in-the-same-order

idx = np.random.permutation(len(x_val))
x_val,y_val = x_val[idx], y_val[idx]

for i in range(len(y_val)):
  print("[",i,"]=",y_val[i],end="  ")

[ 0 ] 2 [ 1 ] 2 [ 2 ] 2 [ 3 ] 4 [ 4 ] 2 [ 5 ] 3 [ 6 ] 3 [ 7 ] 3 [ 8 ] 4 [ 9 ] 2 [ 10 ] 3 [ 11 ] 2 [ 12 ] 0 [ 13 ] 2 [ 14 ] 2 [ 15 ] 2 [ 16 ] 3 [ 17 ] 4 [ 18 ] 2 [ 19 ] 4 [ 20 ] 3 [ 21 ] 3 [ 22 ] 3 [ 23 ] 0 [ 24 ] 4 [ 25 ] 2 [ 26 ] 2 [ 27 ] 2 [ 28 ] 2 [ 29 ] 2 [ 30 ] 2 [ 31 ] 3 [ 32 ] 2 [ 33 ] 2 [ 34 ] 2 [ 35 ] 2 [ 36 ] 2 [ 37 ] 3 [ 38 ] 2 [ 39 ] 2 [ 40 ] 3 [ 41 ] 2 [ 42 ] 2 [ 43 ] 3 [ 44 ] 2 [ 45 ] 3 [ 46 ] 2 [ 47 ] 2 [ 48 ] 4 [ 49 ] 2 [ 50 ] 3 [ 51 ] 2 [ 52 ] 1 [ 53 ] 2 [ 54 ] 4 [ 55 ] 0 [ 56 ] 2 [ 57 ] 2 [ 58 ] 0 [ 59 ] 0 [ 60 ] 3 [ 61 ] 2 [ 62 ] 4 [ 63 ] 4 [ 64 ] 2 [ 65 ] 3 [ 66 ] 2 [ 67 ] 2 [ 68 ] 3 [ 69 ] 2 [ 70 ] 2 [ 71 ] 3 [ 72 ] 2 [ 73 ] 2 [ 74 ] 4 [ 75 ] 2 [ 76 ] 3 [ 77 ] 4 [ 78 ] 0 [ 79 ] 2 [ 80 ] 3 [ 81 ] 2 [ 82 ] 3 [ 83 ] 2 [ 84 ] 4 [ 85 ] 4 [ 86 ] 3 [ 87 ] 3 [ 88 ] 4 [ 89 ] 2 [ 90 ] 2 [ 91 ] 4 [ 92 ] 2 [ 93 ] 4 [ 94 ] 2 [ 95 ] 3 [ 96 ] 2 [ 97 ] 3 [ 98 ] 3 [ 99 ] 4 [ 100 ] 2 [ 101 ] 0 [ 102 ] 2 [ 103 ] 2 [ 104 ] 3 [ 105 ] 1 [ 106 ] 3 [ 107 ] 2 [ 108 ] 0 [ 109 ] 3 [ 110 ] 4 

# TFLITE Interpreter

https://www.tensorflow.org/lite/guide/inference#load_and_run_a_model_in_python
https://www.tensorflow.org/api_docs/python/tf/lite/Interpreter#get_input_details

## lbest_doubleCNNmodel

In [ ]:
def python_predict(data_index):
  # Load TFLite model and allocate tensors.
  interpreter = tf.lite.Interpreter(model_path="best_doubleCNNmodel.tflite")
  interpreter.allocate_tensors()

  # Get input and output tensors.
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()

  # Test model on random input data.
  input_shape = input_details[0]['shape']
  input_data = np.array(x_test[data_index:data_index+1, :], dtype=np.float32)
  print("TEST EEG DATA SAMPLE:")
  print(input_data)

  interpreter.set_tensor(input_details[0]['index'], input_data)
  interpreter.invoke()

  # The function `get_tensor()` returns a copy of the tensor data.
  # Use `tensor()` in order to get a pointer to the tensor.

  output_data = interpreter.get_tensor(output_details[0]['index'])
  print("LABEL:", y_test[data_index])
  print("PREDICTION:", output_data)

In [ ]:
python_predict(3)

TEST EEG DATA SAMPLE:
[[[[-3.7064590e-05]
   [-4.0042542e-05]
   [-4.1466148e-05]
   ...
   [-2.7382621e-06]
   [-2.9561609e-06]
   [-3.7115435e-06]]]]
LABEL: 0
PREDICTION: [[0.69921875 0.1484375  0.109375   0.03515625 0.00390625]]


In [ ]:
python_predict(4)

TEST EEG DATA SAMPLE:
[[[[-9.3716644e-06]
   [-1.0735943e-05]
   [-1.0627420e-05]
   ...
   [-1.0658427e-05]
   [-8.1624166e-06]
   [-6.9376670e-06]]]]
LABEL: 1
PREDICTION: [[0.0390625  0.21875    0.55859375 0.1171875  0.0625    ]]


In [ ]:
python_predict(1)

TEST EEG DATA SAMPLE:
[[[[-8.5924776e-06]
   [-5.7162129e-06]
   [-3.6243839e-06]
   ...
   [ 6.0357975e-06]
   [ 2.3605708e-06]
   [-2.9706873e-06]]]]
LABEL: 2
PREDICTION: [[0.1328125  0.15625    0.62109375 0.08203125 0.0078125 ]]


In [ ]:
python_predict(24)

TEST EEG DATA SAMPLE:
[[[[2.3220753e-05]
   [2.8043580e-05]
   [3.0716474e-05]
   ...
   [4.7872374e-05]
   [4.1887419e-05]
   [3.2953565e-05]]]]
LABEL: 3
PREDICTION: [[0.0390625  0.0390625  0.140625   0.77734375 0.        ]]


In [ ]:
python_predict(0)

TEST EEG DATA SAMPLE:
[[[[ 3.7982758e-07]
   [ 1.1704891e-06]
   [ 1.2014954e-06]
   ...
   [-4.0385748e-06]
   [-5.1703059e-06]
   [-6.6276034e-06]]]]
LABEL: 4
PREDICTION: [[0.06640625 0.2734375  0.375      0.109375   0.171875  ]]


# Calculate Accuracy

In [ ]:
def cal_acc(filename, x, y):
  # Load TFLite model and allocate tensors.
  interpreter = tf.lite.Interpreter(model_path=filename)
  interpreter.allocate_tensors()

  # Get input and output tensors.
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()

  # Test model on random input data.
  print("SAMPLE CORRECT TEST PREDICTIONS: ")
  count = 0
  for i in range(len(x)-1):
    input_data = np.array(x[i:i+1, :], dtype=np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)

    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])

    if np.argmax(output_data) == y[i]:
      print("Prediction:", np.argmax(output_data), "-", "Label:", y[i], end="  |  ")
      count += 1
  
  print()
  return count/(len(x)-1)

## best_doubleCNNmodel

In [ ]:
print("TEST ACCURACY best_doubleCNNmodel: ", cal_acc("best_doubleCNNmodel.tflite", x_test, y_test))

SAMPLE CORRECT TEST PREDICTIONS: 
Prediction: 2 - Label: 2  |  Prediction: 3 - Label: 3  |  Prediction: 1 - Label: 1  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 0 - Label: 0  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Prediction: 2 - Label: 2  |  Predictio

## lfinalbest_doubleCNNmodel_3

In [ ]:
print("Test accuracy for lfinalbest_doubleCNNmodel: ", cal_acc("lfinalbest_doubleCNNmodel_3.tflite", x_test, y_test))

4 - 4,1 - 1,2 - 2,2 - 2,0 - 0,2 - 2,1 - 1,1 - 1,2 - 2,2 - 2,2 - 2,1 - 1,2 - 2,2 - 2,2 - 2,2 - 2,0 - 0,2 - 2,1 - 1,1 - 1,2 - 2,2 - 2,2 - 2,0 - 0,2 - 2,3 - 3,2 - 2,2 - 2,0 - 0,1 - 1,1 - 1,2 - 2,3 - 3,4 - 4,2 - 2,1 - 1,1 - 1,0 - 0,2 - 2,3 - 3,2 - 2,0 - 0,0 - 0,1 - 1,2 - 2,3 - 3,2 - 2,2 - 2,2 - 2,2 - 2,1 - 1,0 - 0,2 - 2,4 - 4,2 - 2,2 - 2,2 - 2,2 - 2,0 - 0,1 - 1,3 - 3,2 - 2,1 - 1,2 - 2,0 - 0,2 - 2,2 - 2,0 - 0,2 - 2,2 - 2,2 - 2,2 - 2,4 - 4,2 - 2,2 - 2,2 - 2,2 - 2,2 - 2,0 - 0,2 - 2,2 - 2,4 - 4,1 - 1,2 - 2,0 - 0,2 - 2,2 - 2,4 - 4,2 - 2,2 - 2,1 - 1,0 - 0,4 - 4,2 - 2,1 - 1,2 - 2,2 - 2,0 - 0,2 - 2,2 - 2,2 - 2,2 - 2,2 - 2,1 - 1,0 - 0,2 - 2,2 - 2,2 - 2,2 - 2,1 - 1,1 - 1,2 - 2,0 - 0,2 - 2,1 - 1,2 - 2,2 - 2,2 - 2,2 - 2,2 - 2,0 - 0,4 - 4,2 - 2,2 - 2,0 - 0,2 - 2,0 - 0,2 - 2,2 - 2,2 - 2,1 - 1,2 - 2,2 - 2,2 - 2,1 - 1,3 - 3,2 - 2,2 - 2,2 - 2,2 - 2,2 - 2,1 - 1,0 - 0,2 - 2,2 - 2,1 - 1,2 - 2,2 - 2,4 - 4,2 - 2,0 - 0,2 - 2,2 - 2,2 - 2,0 - 0,0 - 0,2 - 2,2 - 2,2 - 2,1 - 1,2 - 2,2 - 2,0 - 0,0 - 0,2 - 2,1 - 1,2 - 